## StreamStats API Scraper Automatic

__Description__: Tool to automatically run the [USGS StreamStats tool](https://www.usgs.gov/mission-areas/water-resources/science/streamstats-streamflow-statistics-and-spatial-analysis-tools?qt-science_center_objects=0#qt-science_center_objects) for multiple points within a catchment and return the flow frequency curves and subcatchment boundaries.

__Input__: A shapefile containing the latitude and longitude of points on the stream grid for the specified state (confluence and main stem locations).

__Output__: GeoJSON file containing the delinated catchment boundary and flow frequency data for each point, as well as a CSV file containing the flow frequency data.

*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [17]:
import os
import re
import sys
sys.path.append('../USGStools')
from StreamStats_API_Scraper import*
import geopandas as gpd
from geojson import dump

### Specify the state abbreviation and location of the shapefile: 

##### Specify:

In [2]:
state='WI' #The state abbreviation in uppercase

path=r'C:\Users\sputnam\Documents\GitHub\usgs-tools\StreamStats\results\Kewaunee_WI' #Specify the location of the shapefile containing the lat/lon of points on the stream grid

name='Confluences1.shp' #The name of the shapefile

use_epsg='4326' #Specify a consistent coordinate reference system

allresults=os.path.join(path,'AllStreamStats') #Location to save the StreamStats results for each polygon

if os.path.isdir(allresults)==False: #If the desired path does not exist, create it.
    os.mkdir(allresults)    

##### Load the shapefile:

In [3]:
gdf=gpd.read_file(os.path.join(path, name)) #Read the shapefile as a geopandas dataframe

gdf=gdf.set_index('num').copy(deep=True) #Set the index to the confluence number

gdf=gdf.to_crs({'init': 'epsg:{0}'.format(use_epsg)}) #Transform the coordinate reference system of the geodataframe

geom=gdf.geometry #Extract the shapley geometry for the outlets in the shapefile

print(geom.head(2))

num
0    POINT (-87.43693610883072 44.62174776562063)
1    POINT (-87.44794369555855 44.61893847649607)
Name: geometry, dtype: object


### Run the API tool for each point:

In [4]:
polyg={} #Dictionary to store the catchment polygons (catchment boundaries) 

ffdata={} #Dictionary to store the outlet flow frequency data dictionaries

if state=='WI': get_flow=False 
print_status=True

for i, xy in enumerate(geom): #For gdf.geometry:
    lon, lat = xy.x, xy.y #Longitude and latitude for each shapely point
    if print_status==True:
        print("Lat/Lon:", lat, lon)
    polyg[i], ff_json  = SS_scrape(state, lon, lat, use_epsg, print_status) #Run the SS_scrape function. Option: set status=False to hide print statements
    if get_flow: 
        ffdata[i]= get_peaks(ff_json) #Use the function above to extract the json data
        polyg[i]['features'][0]['ffcurve']=ffdata[i]
    with open(os.path.join(allresults,'StreamStats_Polygons_{0}.geojson'.format(gdf.index[i])), 'w') as f:
       dump(polyg[i], f)       

Lat/Lon: 44.62174776562063 -87.43693610883072
Fetched Peak Flows
Lat/Lon: 44.61893847649607 -87.44794369555855
Fetched Peak Flows
Lat/Lon: 44.61733078041878 -87.4439636656148
Fetched Peak Flows
Lat/Lon: 44.61630613512092 -87.44485448366903
Fetched Peak Flows
Lat/Lon: 44.61137850788752 -87.44814234295116
Fetched Peak Flows
Lat/Lon: 44.61023281626847 -87.44714673613238
Fetched Peak Flows
Lat/Lon: 44.61223172253167 -87.44879475329016
Fetched Peak Flows
Lat/Lon: 44.61491013410794 -87.45682264909334
Fetched Peak Flows
Lat/Lon: 44.613788682486 -87.45620412197006
Fetched Peak Flows
Lat/Lon: 44.61003057024509 -87.46086408663705
Fetched Peak Flows
Lat/Lon: 44.611659297149366 -87.46940530632472
Fetched Peak Flows
Lat/Lon: 44.61860761074862 -87.47232940506365
Fetched Peak Flows
Lat/Lon: 44.622704213198496 -87.47295214736857
Fetched Peak Flows
Lat/Lon: 44.608799164134155 -87.47965537960289
Fetched Peak Flows
Lat/Lon: 44.62658083465956 -87.4743636263028
Fetched Peak Flows
Lat/Lon: 44.62582424896232

### Load the results:

In [10]:
poly_files=[] #Empty list to store the geojson paths

poly_files=load_all_results(allresults)

gdf=gpd.GeoDataFrame(crs={'init': 'epsg:{}'.format(use_epsg)})
                          
for _,filename in enumerate(poly_files):
    num=re.findall('\d+', filename)
    temp_df=gpd.read_file(filename)
    #temp_df['ID_Name']='Algoma'
    temp_df['ID_Num']=int(num[0])
    gdf=gdf.append(temp_df.iloc[0])

###  Save:

##### The flow frequency data as a CSV:

In [11]:
if get_flow: ffdata=ff_summary(pp_dic) #Run this function to construct the summary table for all outlet locations
    
if get_flow: ffdata.to_csv(os.path.join(path,'StreamStats_FlowFrequency.csv')) #Save the results as a csv

##### The catchment polygons as a Shapefile:

In [15]:
gdf.to_file(filename = os.path.join(path,'StreamStats_Polygons.shp')) #Export the geodataframe as a shapefule

##### The catchment polygons as a geojson:

In [16]:
with open(os.path.join(path,'StreamStats_Polygons.geojson'), 'w') as f:
     dump(gdf, f)  

# END